<a href="https://colab.research.google.com/github/Sim-mi-gyeong/DeepLearning/blob/main/Triplet_Loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U tensorflow-addons

     |████████████████████████████████| 706kB 4.6MB/s 


In [ ]:
import io #  다양한 유형의 I/O를 처리하기 위한 파이썬의 주 장치를 제공

In [ ]:
import numpy as np

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

In [ ]:
def _normalize_img(img, label):
  img=tf.cast(img, tf.float32) / 255.
  return (img, label)

train_dataset, test_dataset = tfds.load(name='mnist', split=['train', 'test'], as_supervised=True)  # as_supervised=True: 튜플 형태로 데이터 리턴 (input, label)

# Build your input pipelines
train_dataset=train_dataset.shuffle(1024).batch(32)
train_dataset=train_dataset.map(_normalize_img)

test_dataset=test_dataset.batch(32)
test_dataset=test_dataset.map(_normalize_img)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
model=tf.keras.Sequential([
                           tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1)),
                           tf.keras.layers.MaxPooling2D(pool_size=2),
                           tf.keras.layers.Dropout(0.3),
                           tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
                           tf.keras.layers.MaxPooling2D(pool_size=2),
                           tf.keras.layers.Dropout(0.3),
                           tf.keras.layers.Flatten(),
                           tf.keras.layers.Dense(256, activation=None),  # No activation on final dense layer
                           tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) #L2 normalize embeddings
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tfa.losses.TripletSemiHardLoss())

In [ ]:
hitory=model.fit(train_dataset, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 121s 63ms/step - loss: 0.7055
Epoch 2/5
1875/1875 [==============================] - 114s 61ms/step - loss: 0.4703
Epoch 3/5
1875/1875 [==============================] - 92s 49ms/step - loss: 0.4257
Epoch 4/5
1875/1875 [==============================] - 85s 45ms/step - loss: 0.4053
Epoch 5/5
1875/1875 [==============================] - 84s 45ms/step - loss: 0.3929


In [ ]:
# Evaluate the network
results=model.predict(test_dataset)

In [ ]:
np.savetxt("vecs.tsv", results, delimiter='\t')

out_m=io.open('meta.tsv', 'w', encoding='utf-8')
for img, labels in tfds.as_numpy(test_dataset):
  [out_m.write(str(x) + '\n') for x in labels]
out_m.close()

try:
  from google.colab import files
  files.download('vecs.tsv')
  files.download('meta.tsv')
except:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>